# Market Basket Analysis

# Geração de Regras de Associação - Algoritmo Apriori

### Recomendação de Produtos

## Como podemos definir o que são regras de associação?

Regras de associação são um conjunto de métricas e regras criadas para identificar a relação entre dados. Esse tipo de técnica é muito utilizado em sistemas de recomendação, como por exemplo a Netflix, Spotify e também muito utilizado em cestas de compras em supermercados, lojas virtuais, etc.

Basicamente as regras de associações funcionam com algumas métricas de suporte a decisão. Essas métricas direcionam a identificação sobre qual a probabilidade de um produto "X" está relacionado com o produto "Y" por exemplo.

### Algoritmo Apriori

O Algoritmo Apriori é uma das técnicas de associação mais utilizada para criar associação aos dados. Vamos implementar o algoritmo passo a passo em Python e entender seu funcionamento.

Basicamente, o algoritmo utiliza 3 métricas para associar dados, sendo elas:
* Suporte
* Confiança
* Lift

https://pypi.org/project/apyori/

Vamos imaginar que temos dois tipos de produtos, X e Y:

### Support: 

Suporte: Frequência relativa da associação: (XY/n) - número de clientes com os dois produtos dividido pelo número total de clientes. 

### Confidence: 
Confiança: (XY/X) - Percentual de clientes que compraram  produto Y dentro daqueles que compraram X. Proporção de vezes tanto para o produto X quanto para Y dentro de uma **mesma** transação. É a probabilidadepara uma nova transação que contém os itens no LHS da regra. 

### Lift:

O lift traz o valor de frequência sobre um dado correlacionado a outro dado. Quanto aumenta a frequência de Y em relação a X?

### Implementando o algoritmo de Apriori

Para implementar o algoritmo, vamos utilizar uma base de dados que contém linhas de registros de filmes assistidos por usuário.

#### Importando as bibliotecas necessárias:


In [ ]:
! pip install apyori
! pip install efficient-apriori
! pip install missingno

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import missingno as msno 
from apyori import apriori

#### Importando a base de dados para trabalharmos com o algoritmo:

In [ ]:
filmes_dataset = pd.read_csv('movie_dataset.csv', header = None)
total_registros = len(filmes_dataset)
print(total_registros)

In [ ]:
filmes_dataset.head(5)

In [ ]:
msno.matrix(filmes_dataset)

#### Processamento dos dados:

A biblioteca Apyori que vamos usar requer que nosso conjunto de dados esteja na forma de uma lista de produtos/filmes. Vamos realizar a conversão do dataframe "filmes_dataset" em uma grande lista:

In [ ]:
registros = []  
for i in range(0, total_registros):  
    registros.append([str(filmes_dataset.values[i,j]) for j in range(0, 20)])

In [ ]:
#registros # analisando o resultado da lista

#### Especificando os parâmetros do algoritmo apriori:

- min_support (tamanho mínimo para suporte)
- min_confidence (Confiança mínima)
- min_lift (tamanho mínimo de lift)
- min_length (tamanho mínimo de regras)

### Aplicando o algoritmo:

In [ ]:
rules = apriori(registros, min_support=0.005, min_confidence=0.20, min_lift=3, min_length=2)
# opções: min_support=0.003, min_confidence=0.20, min_lift=3, min_length=2, max_lenght =2
results = list(rules)  #Convertendo regras encontradas em lista para visualizar os dados.

Número total de regras extraídas pela classe a priori:

In [ ]:
print(len(results))

In [ ]:
results

Temos um total de 10 regras, vamos analisar quais são elas:

In [ ]:
for regras in results:
  print(results)

Vamos analisar o primeiro item trazido pela regra:

**{'Jumanji', 'Kung Fu Panda'}**

- Temos um suporte de 0.0159, ou seja foi calculado o número total de transações contendo o filme Jumanji e Kung Fu Panda dividido pelo total de transações da base.

- O nível de confiança para a regra é de 0.32, ou seja 30% das vezes que alguém assistiu Jumanji, também assistiu Kung Fu Panda.

- O lift é de 3.29, ou seja, quem assistiu Jumanji tem 3 x mais chances de assistir Kung Fu Panda.



O código abaixo organiza os resultados em tabela para identificarnos melhor as regras criadas:

In [ ]:
# opção 1 

def inspect(results):
    lhs = [tuple(result[2][0][0])[0] for result in results]
    rhs = [tuple(result[2][0][1])[0] for result in results]
    supports = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts = [result[2][0][3] for result in results]   
    return list(zip(lhs, rhs, supports,confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results),columns = ['Left Hand Side', 'Right Hand Side','Support','Confidence', 'Lift'])

In [ ]:
resultsinDataFrame

In [ ]:
resultsinDataFrame.nlargest(n=10, columns = 'Lift')


In [ ]:
# opção 2

resultados = []
for item in results:
    
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    value0 = str(items[0])
    value1 = str(items[1])

    #second index of the inner list
    value2 = str(item[1])[:7]

    #third index of the list located at 0th
    #of the third index of the inner list

    value3 = str(item[2][0][2])[:7]
    value4 = str(item[2][0][3])[:7]
    
    linhas = (value0, value1,value2,value3,value4)
    resultados.append(linhas)
    
headers = ['Title 1','Title 2','Support','Confidence','Lift']
sugestao_filmes = pd.DataFrame.from_records(resultados, columns = headers)

print(sugestao_filmes)